In [ ]:
# This takes... 

In [2]:
import xarray as xr 
import numpy as np

# This Script is to Convert the output .txt files into .netcdf files
- I would like two variables in this data set 
- Whether there is a block 
- Where the block is in the plane 

In [4]:
class block:
    
    def __init__(self, time_index, list_of_coord_indexes,length):
        self.time_index = time_index      #this is in days past november 30th
        self.list_of_coord_indexes = list_of_coord_indexes 
        self.length = length              #this is in days
    

In [21]:
for year in np.arange(1980,2018,1):

    f = open(f"out_data/{year}/EUR_LSB_djf_{year}.txt", "r")
    lines = f.readlines()
    start = [i for i,x in enumerate(lines) if x=='Start Index:\n'] 
    k=0
    block_list = []
    while k < len(start) :
        if k == len(start) -1:
            block_lines = lines[int(start[k]):len(lines) -1]
        else:
            block_lines = lines[int(start[k]):int(start[k+1])]
        begin_index = block_lines[1]   #saving the start index
        length = block_lines[3]        #saving the length 
        days = []
        l = 5
        while l < len(block_lines):
            if 'Day 0:' in block_lines[l]:
                all_coords = []
                pass
            elif 'y' in block_lines[l]: #if we see day that isnt day 1
                days.append(all_coords)
                all_coords = []
                pass 
            elif '---' in block_lines[l]:
                days.append(all_coords)
                pass
            elif block_lines[l] == '\n':
                pass
            else:
                coord = [int(s) for s in block_lines[l].split(',') if s.isdigit()]
                coord.append(int(block_lines[l].split(',')[1].split('/')[0]))
                if coord == list():
                    del(coord)
                else:
                    all_coords.append(coord)
            
            l+=1
        days.append(all_coords)
        
        k+=1
        block_list.append(block(begin_index,days,length))

    da = xr.open_dataset(f'data/{year}djf_final.nc')['Z'].isel(p = 15) #this selects the 500 hPa pressure level 
    lon = da.__getitem__('lon').values
    lat = da.__getitem__('lat').values
    time_orig = da.__getitem__('t').values

    k=0 
    time_fixed = []
    while k<len(time_orig):
        if k%4 ==0:
            time_fixed.append(time_orig[k])
        else:
            pass
        k+=1

    block_array = np.zeros((len(time_fixed),len(lat),len(lon)))
    for blk in block_list:
        days = blk.list_of_coord_indexes
        ind = blk.time_index
        length = blk.length
        print(len(days))
        for d in range(len(days)): #cycling through block days
            for coord in days[d]:
                block_array[int(ind)+int(d),coord[0],coord[1]] = 1    

    output_da_grid = xr.DataArray(block_array, name = 'TM90_grid_index', coords = {"t": time_fixed, 'lat': da['lat'], 'lon': da['lon']}, dims = ['t','lat','lon']).to_dataset()
    output_da_grid.to_netcdf(f'out_data/{year}/EUR_LSB{year}_grid_TM90_fin.nc')        
    new = np.zeros(len(output_da_grid['t'].values))
    
    for k in range(len(new)):
        if np.any(output_da_grid['TM90_grid_index'][k]) == 1:
            new[k] = 1
    no_grid = xr.DataArray(new, name = 'TM90_index_linear', coords = {"t": time_fixed}, dims = ['t'])        
    no_grid.to_dataset().to_netcdf(f'out_data/{year}/EUR_{year}_linear_TM90_fin.nc')   

    
    
    
    




7
7
5
8
7
7
6
6
7
6
10
6
6
6
7
6
6
5
6
7
6
6
5
6
5
7
7
7
9
8
7
5
5
9
5
7
6
6
7
5
5
8
5
7
7
5
7
8
6
7
7
7
9
6
9
6
6
6
6
6
5
6
6
6
7
8
7
8
5
7
5
8
6
9
5


In [9]:
xr.open_dataset('data/1980djf_final.nc')


TypeError: unhashable type: 'slice'

# Now to create a netcdf file from blocking list
- requires us to load the input data

In [153]:
da = xr.open_dataset(f'data/{year}djf_final.nc')['Z'].isel(p = 15) #this selects the 500 hPa pressure level 
lon = da.__getitem__('lon').values
lat = da.__getitem__('lat').values
time_orig = da.__getitem__('t').values

k=0 
time_fixed = []
while k<len(time_orig):
    if k%4 ==0:
        time_fixed.append(time_orig[k])
    else:
        pass
    k+=1
    
block_array = np.zeros((len(time_fixed),len(lat),len(lon)))
for blk in block_list:
    days = blk.list_of_coord_indexes
    ind = blk.time_index
    length = blk.length
    for d in range(len(days)): #cycling through block days
        for coord in days[d]:
            block_array[int(ind)+int(d),coord[0],coord[1]] = 1
ds = output_da_grid[:,12:31,14:35].to_dataset()
new = np.zeros(len(ds['t'].values))
print(len(new))
print(len(time_fixed))
for k in range(len(new)):
    if np.any(ds.to_array().values[0][k]) == 1:
        new[k] = 1
no_grid = xr.DataArray(new, name = 'TM90_index_linear', coords = {"t": time_fixed}, dims = ['t'])        
no_grid.to_dataset().to_netcdf('linear_TM90.nc')   



91
91


In [114]:



output_da_grid = xr.DataArray(block_array, name = 'TM90_grid_index', coords = {"t": time_fixed, 'lat': da['lat'], 'lon': da['lon']}, dims = ['t','lat','lon'])



In [123]:
output_da_grid.to_dataset()

<xarray.Dataset>
Dimensions:          (t: 90, lat: 36, lon: 36)
Coordinates:
  * t                (t) datetime64[ns] 2014-12-01 2014-12-02 ... 2015-02-28
  * lat              (lat) float64 0.0 2.5 5.0 7.5 10.0 ... 80.0 82.5 85.0 87.5
  * lon              (lon) float64 -45.0 -42.5 -40.0 -37.5 ... 37.5 40.0 42.5
Data variables:
    TM90_grid_index  (t, lat, lon) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

# Need to select the right region with indexing
- 30-75 lat
- -10-40 lon 


In [128]:
ds = output_da_grid[:,12:31,14:35].to_dataset()

In [130]:
ds.to_netcdf('final_output_grid_limited_TM90.nc')
output_da_grid.to_netcdf('final_output_grid_TM90.nc')

In [143]:
ds.to_array()[0][0]

<xarray.DataArray (lat: 19, lon: 21)>
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]])
Coordinates:
    t         datetime64[ns] 2014-12-01
  * lat       (lat) float64 30.0 32.5 35.0 37.5 40.0 ... 67.5 70.0 72.5 75.0
  * lon       (lon) float64 -10.0 -7.5 -5.0 -2.5 0.0 ... 32.5 35.0 37.5 40.0
    variable  <U15 'TM90_grid_index'

# Now just a 1D index for the region

In [144]:
new = np.zeros(len(ds['t'].values))
for k in range(len(new)):
    if np.any(ds.to_array().values[0][k]) == 1:
        new[k] = 1
        
    

In [150]:
no_grid = xr.DataArray(new, name = 'TM90_index_linear', coords = {"t": time_fixed}, dims = ['t'])

In [151]:
no_grid.to_dataset().to_netcdf('linear_TM90.nc')

In [ ]:

for year in np.arange(1980,2020,1):

    f = open(f"out_data/{year}/LSB_djf_{year}.txt", "r")

In [3]:
year = 1980
filename = '~/Atmospheric_blocking/data/1980djf_final.nc'
data = xr.open_dataset(filename)
data = data.isel(p=15)
print(data)

<xarray.Dataset>
Dimensions:  (t: 364, lon: 36, lat: 36)
Coordinates:
  * t        (t) datetime64[ns] 1979-12-01 ... 1980-02-29T18:00:00
  * lon      (lon) float64 -45.0 -42.5 -40.0 -37.5 -35.0 ... 35.0 37.5 40.0 42.5
  * lat      (lat) float64 0.0 2.5 5.0 7.5 10.0 ... 77.5 80.0 82.5 85.0 87.5
    p        float32 500.0
Data variables:
    Z        (t, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.8 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Fri Jan 21 16:21:13 2022: cdo -remapbil,~/Atmospheric_block...
    CDO:          Climate Data Operators version 1.9.8 (https://mpimet.mpg.de...


In [4]:
year = 1980
filename = '~/Atmospheric_blocking/data/1980djf_final.nc'
data = xr.open_dataset(filename)
data = data.isel(p=15)
print(data)

<xarray.Dataset>
Dimensions:  (t: 364, lon: 36, lat: 36)
Coordinates:
  * t        (t) datetime64[ns] 1983-04-08T01:31:13.828125 ... 1979-12-01
  * lon      (lon) float64 -45.0 -42.5 -40.0 -37.5 -35.0 ... 27.5 30.0 32.5 35.0
  * lat      (lat) float64 37.5 40.0 42.5 45.0 ... 3.162e+22 3.08e+22 2.722e+22
    p        float32 1.223e+03
Data variables:
    Z        (t, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.8 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Fri Jan 21 16:21:13 2022: cdo -remapbil,~/Atmospheric_block...
    CDO:          Climate Data Operators version 1.9.8 (https://mpimet.mpg.de...


/opt/jaspy/lib/python3.8/site-packages/xarray/core/indexing.py:419: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
